In [1]:
import sklearn
print(sklearn.__version__)

0.20.3


In [2]:
from sklearn.impute import SimpleImputer

In [19]:
test=pd.read_csv(r'C:\Users\user\test_LoanPrediction.csv')
#train=pd.read_csv('train_LoanPrediction')

In [9]:
import os
print(os.getcwd())

C:\Users\user


In [17]:
import os
print (os.path.abspath("test_LoanPrediction.csv"))

C:\Users\user\test_LoanPrediction.csv


In [23]:
import pandas as pd
test=pd.read_csv(r'C:\Users\user\test_LoanPrediction.csv')
train=pd.read_csv(r'C:\Users\user\train_LoanPrediction.csv')
train=train.drop("Loan_ID", axis=1)

In [24]:
def describe_data(df):
    print("Data Types:")
    print(df.dtypes)
    print()
    print("Rows and Columns:")
    print(df.shape)
    print()
    print("Null Values:")
    print(df.apply(lambda x: sum(x.isnull())/len(df)))

describe_data(train)

Data Types:
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

Rows and Columns:
(614, 12)

Null Values:
Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64


In [32]:
numeric_features=train.select_dtypes(include=['int64', 'float64']).columns
categorical_features=train.select_dtypes(include=['object']).drop(['Loan_Status'],axis=1).columns

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.pipeline import Pipeline

In [26]:
numeric_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [28]:
categorical_transformer=Pipeline(steps=[
    ('imputer' , SimpleImputer(strategy='constant' , fill_value='missing')), 
    ('onehot' , OneHotEncoder(handle_unknown='ignore'))])

In [29]:
from sklearn.compose import ColumnTransformer

In [33]:
preprocessor= ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features),
                  ('cat', categorical_transformer, categorical_features)])

In [34]:
from sklearn.model_selection import train_test_split
X=train.drop('Loan_Status',axis=1)
y=train['Loan_Status']

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)


In [37]:
from sklearn.linear_model import LogisticRegression
lr=Pipeline(steps=[('preprocessor', preprocessor),
                   ('classifier', LogisticRegression(solver='lbfgs'))])
lr.fit(X_train, y_train)
print("model score:%.3f"% lr.score(X_test, y_test))

model score:0.797


In [38]:
test_no_id=test.drop('Loan_ID',axis=1)

In [39]:
test_predictions=lr.predict(test_no_id)

In [40]:
Loan_ID=test['Loan_ID']
submission_df_1=pd.DataFrame({
    "Loan_ID":Loan_ID,
    "Loan_Status":test_predictions
})

In [41]:
submission_df_1.to_csv("submission_1.csv", index=False)